# 03. Streamlit UI 개발

**Rerank 적용 멀티모달 RAG 챗봇**의 웹 UI를 Streamlit으로 구현합니다.

## 🎯 목표
- 직관적인 채팅 인터페이스
- 파일 업로드 기능 (텍스트, 이미지, PDF)
- 실시간 대화 및 세션 관리
- 소스 문서 표시 및 추적
- **Rerank 기반 고품질 답변 제공**

## 🚀 핵심 기능
- **멀티모달 문서 처리**: Claude 3.7 Sonnet 파싱
- **벡터 검색**: Titan Text Embeddings v2
- **Rerank 최적화**: Cohere Rerank v3-5로 검색 정확도 향상
- **고품질 답변**: Claude 3.7 Sonnet 생성

## 📋 개발 단계
1. 기본 설정 및 import
2. 페이지 레이아웃 설계
3. 사이드바 구성
4. 채팅 인터페이스
5. 파일 업로드 기능
6. **Rerank RAG 연동**
7. 최종 app.py 생성

## 1. 기본 설정 및 Import

In [6]:
# 필요한 라이브러리 import
import streamlit as st
import boto3
import os
from datetime import datetime
import json
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

print("✅ 라이브러리 import 완료")
print(f"Streamlit 버전: {st.__version__}")

✅ 라이브러리 import 완료
Streamlit 버전: 1.48.1


## 2. 환경 변수 및 설정

In [7]:
# 환경 변수 확인
kb_id = os.getenv('BEDROCK_KNOWLEDGE_BASE_ID')
model_id = os.getenv('BEDROCK_MODEL_ID')
region = os.getenv('AWS_REGION')
account_id = os.getenv('AWS_ACCOUNT_ID')
s3_bucket = os.getenv('S3_BUCKET_NAME')

print("🔍 환경 변수 확인:")
print(f"Knowledge Base ID: {kb_id}")
print(f"Model ID: {model_id}")
print(f"Region: {region}")
print(f"Account ID: {account_id}")
print(f"S3 Bucket: {s3_bucket}")

if not all([kb_id, model_id, region, account_id]):
    print("⚠️ 일부 환경 변수가 설정되지 않았습니다.")
else:
    print("✅ 모든 환경 변수 설정 완료")

🔍 환경 변수 확인:
Knowledge Base ID: SVJFXS8QCG
Model ID: anthropic.claude-3-7-sonnet-20250219-v1:0
Region: us-west-2
Account ID: 191629784099
S3 Bucket: my-rag-chatbot-5711d275
✅ 모든 환경 변수 설정 완료


## 3. Bedrock 클라이언트 설정

In [8]:
# Bedrock 클라이언트 초기화
def initialize_bedrock_client():
    """Bedrock 클라이언트 초기화"""
    try:
        bedrock_agent_runtime = boto3.client(
            'bedrock-agent-runtime',
            region_name=region
        )
        
        bedrock_runtime = boto3.client(
            'bedrock-runtime',
            region_name=region
        )
        
        return bedrock_agent_runtime, bedrock_runtime
    except Exception as e:
        print(f"❌ Bedrock 클라이언트 초기화 실패: {e}")
        return None, None

# 클라이언트 초기화 테스트
bedrock_agent, bedrock_runtime = initialize_bedrock_client()

if bedrock_agent and bedrock_runtime:
    print("✅ Bedrock 클라이언트 초기화 성공")
else:
    print("❌ Bedrock 클라이언트 초기화 실패")

✅ Bedrock 클라이언트 초기화 성공


## 4. RAG 검색 및 생성 함수

In [11]:
def retrieve_and_generate(query, session_id=None):
    """Knowledge Base를 사용한 RAG 검색 및 답변 생성"""
    try:
        # RetrieveAndGenerate API 호출
        request_params = {
            'input': {
                'text': query
            },
            'retrieveAndGenerateConfiguration': {
                'type': 'KNOWLEDGE_BASE',
                'knowledgeBaseConfiguration': {
                    'knowledgeBaseId': kb_id,
                    'modelArn': f'arn:aws:bedrock:{region}:{account_id}:inference-profile/us.{model_id}',
                    'retrievalConfiguration': {
                        'vectorSearchConfiguration': {
                            'numberOfResults': 5,
                            'overrideSearchType': 'HYBRID'
                        }
                    },
                    'generationConfiguration': {
                        'inferenceConfig': {
                            'textInferenceConfig': {
                                'maxTokens': 2048,
                                'temperature': 0.1,
                                'topP': 0.9
                            }
                        }
                    }
                }
            }
        }
        
        # 세션 ID가 있으면 추가
        if session_id:
            request_params['sessionId'] = session_id
        
        response = bedrock_agent.retrieve_and_generate(**request_params)
        
        # 응답 파싱
        answer = response['output']['text']
        session_id = response.get('sessionId', session_id)
        citations = response.get('citations', [])
        
        return {
            'answer': answer,
            'session_id': session_id,
            'citations': citations,
            'success': True
        }
        
    except Exception as e:
        print(f"❌ RAG 검색 실패: {e}")
        return {
            'answer': f"죄송합니다. 오류가 발생했습니다: {str(e)}",
            'session_id': session_id,
            'citations': [],
            'success': False
        }

# 테스트 쿼리
if bedrock_agent:
    test_question = "안녕하세요, 테스트입니다."
    test_result = retrieve_and_generate(test_question)
    print(f"✅ RAG 테스트 결과: {test_result['success']}")
    if test_result['success']:
        print(f"질문: {test_question}")
        print(f"답변: {test_result['answer'][:100]}...")
else:
    print("⚠️ Bedrock 클라이언트가 없어 RAG 테스트를 건너뜁니다.")

✅ RAG 테스트 결과: True
질문: 안녕하세요, 테스트입니다.
답변: 안녕하세요! 어떤 질문이 있으신가요? 검색 결과를 바탕으로 도움을 드릴 수 있습니다. 검색 증강 생성(RAG), AWS 서비스, 또는 다른 주제에 관해 궁금한 점이 있으시면 말씀해...


## 5. Streamlit 앱 구조 설계

In [7]:
# Streamlit 앱의 기본 구조를 문자열로 정의
streamlit_app_code = '''
import streamlit as st
import boto3
import os
from datetime import datetime
import json
from dotenv import load_dotenv
import uuid

# 페이지 설정
st.set_page_config(
    page_title="멀티모달 RAG 챗봇",
    page_icon="🤖",
    layout="wide",
    initial_sidebar_state="expanded"
)

# 환경 변수 로드
load_dotenv()

# 전역 변수
KB_ID = os.getenv('BEDROCK_KNOWLEDGE_BASE_ID')
MODEL_ID = os.getenv('BEDROCK_MODEL_ID')
REGION = os.getenv('AWS_REGION')
ACCOUNT_ID = os.getenv('AWS_ACCOUNT_ID')
S3_BUCKET = os.getenv('S3_BUCKET_NAME')

# 세션 상태 초기화
if "messages" not in st.session_state:
    st.session_state.messages = []
if "session_id" not in st.session_state:
    st.session_state.session_id = str(uuid.uuid4())
if "bedrock_client" not in st.session_state:
    st.session_state.bedrock_client = None
'''

print("📝 Streamlit 앱 기본 구조 정의 완료")
print("다음 단계: 사이드바 및 메인 인터페이스 구현")

📝 Streamlit 앱 기본 구조 정의 완료
다음 단계: 사이드바 및 메인 인터페이스 구현


## 6. 사이드바 구성

In [8]:
# 사이드바 코드 추가
sidebar_code = '''
# 사이드바 구성
def setup_sidebar():
    """사이드바 설정 및 구성"""
    with st.sidebar:
        st.title("🤖 RAG 챗봇 설정")
        
        # 환경 상태 표시
        st.subheader("📊 시스템 상태")
        
        if KB_ID:
            st.success(f"✅ Knowledge Base: {KB_ID[:8]}...")
        else:
            st.error("❌ Knowledge Base ID 없음")
            
        if MODEL_ID:
            st.success(f"✅ Model: {MODEL_ID.split('.')[-1]}")
        else:
            st.error("❌ Model ID 없음")
            
        if REGION:
            st.success(f"✅ Region: {REGION}")
        else:
            st.error("❌ AWS Region 없음")
        
        st.divider()
        
        # 채팅 설정
        st.subheader("⚙️ 채팅 설정")
        
        # 온도 설정
        temperature = st.slider(
            "응답 창의성 (Temperature)",
            min_value=0.0,
            max_value=1.0,
            value=0.1,
            step=0.1,
            help="낮을수록 일관된 답변, 높을수록 창의적 답변"
        )
        
        # 최대 토큰 수
        max_tokens = st.slider(
            "최대 응답 길이 (Tokens)",
            min_value=256,
            max_value=4096,
            value=2048,
            step=256,
            help="응답의 최대 길이를 제한합니다"
        )
        
        # 검색 결과 수
        num_results = st.slider(
            "검색 결과 수",
            min_value=1,
            max_value=10,
            value=5,
            help="Knowledge Base에서 검색할 문서 수"
        )
        
        st.divider()
        
        # 세션 관리
        st.subheader("💬 세션 관리")
        
        col1, col2 = st.columns(2)
        
        with col1:
            if st.button("🗑️ 채팅 초기화", use_container_width=True):
                st.session_state.messages = []
                st.session_state.session_id = str(uuid.uuid4())
                st.rerun()
        
        with col2:
            if st.button("🔄 새 세션", use_container_width=True):
                st.session_state.session_id = str(uuid.uuid4())
                st.success("새 세션이 시작되었습니다!")
        
        # 현재 세션 ID 표시
        st.caption(f"세션 ID: {st.session_state.session_id[:8]}...")
        
        return {
            'temperature': temperature,
            'max_tokens': max_tokens,
            'num_results': num_results
        }
'''

print("📱 사이드바 구성 코드 완료")
print("포함된 기능:")
print("- 시스템 상태 표시")
print("- 채팅 설정 (온도, 토큰 수, 검색 결과 수)")
print("- 세션 관리 (초기화, 새 세션)")

📱 사이드바 구성 코드 완료
포함된 기능:
- 시스템 상태 표시
- 채팅 설정 (온도, 토큰 수, 검색 결과 수)
- 세션 관리 (초기화, 새 세션)


## 7. 메인 채팅 인터페이스

In [9]:
# 메인 채팅 인터페이스 코드
chat_interface_code = '''
# Bedrock 클라이언트 초기화
@st.cache_resource
def get_bedrock_client():
    """Bedrock 클라이언트 초기화 (캐시됨)"""
    try:
        return boto3.client('bedrock-agent-runtime', region_name=REGION)
    except Exception as e:
        st.error(f"Bedrock 클라이언트 초기화 실패: {e}")
        return None

def retrieve_and_generate(query, config):
    """Knowledge Base를 사용한 RAG 검색 및 답변 생성"""
    client = get_bedrock_client()
    if not client:
        return {
            'answer': "Bedrock 클라이언트를 초기화할 수 없습니다.",
            'citations': [],
            'success': False
        }
    
    try:
        request_params = {
            'input': {'text': query},
            'retrieveAndGenerateConfiguration': {
                'type': 'KNOWLEDGE_BASE',
                'knowledgeBaseConfiguration': {
                    'knowledgeBaseId': KB_ID,
                    'modelArn': f'arn:aws:bedrock:{REGION}::foundation-model/{MODEL_ID}',
                    'retrievalConfiguration': {
                        'vectorSearchConfiguration': {
                            'numberOfResults': config['num_results'],
                            'overrideSearchType': 'HYBRID'
                        }
                    },
                    'generationConfiguration': {
                        'inferenceConfig': {
                            'textInferenceConfig': {
                                'maxTokens': config['max_tokens'],
                                'temperature': config['temperature'],
                                'topP': 0.9
                            }
                        }
                    }
                }
            },
            'sessionId': st.session_state.session_id
        }
        
        response = client.retrieve_and_generate(**request_params)
        
        return {
            'answer': response['output']['text'],
            'citations': response.get('citations', []),
            'success': True
        }
        
    except Exception as e:
        return {
            'answer': f"오류가 발생했습니다: {str(e)}",
            'citations': [],
            'success': False
        }

def display_chat_messages():
    """채팅 메시지 표시"""
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])
            
            # 소스 문서 표시 (assistant 메시지에만)
            if message["role"] == "assistant" and "citations" in message:
                citations = message["citations"]
                if citations:
                    with st.expander(f"📚 참고 문서 ({len(citations)}개)"):
                        for i, citation in enumerate(citations, 1):
                            if 'retrievedReferences' in citation:
                                for ref in citation['retrievedReferences']:
                                    location = ref.get('location', {})
                                    s3_location = location.get('s3Location', {})
                                    uri = s3_location.get('uri', 'Unknown')
                                    
                                    st.markdown(f"**{i}. {uri.split('/')[-1]}**")
                                    
                                    content = ref.get('content', {}).get('text', '')
                                    if content:
                                        st.markdown(f"```\n{content[:300]}...\n```")
                                    
                                    st.markdown("---")

def main_chat_interface(config):
    """메인 채팅 인터페이스"""
    st.title("🤖 멀티모달 RAG 챗봇")
    st.markdown("**Rerank 기반 고품질 답변 제공** | Claude 3.7 Sonnet + Cohere Rerank v3-5")
    
    # 환경 변수 확인
    if not all([KB_ID, MODEL_ID, REGION]):
        st.error("⚠️ 환경 변수가 올바르게 설정되지 않았습니다. .env 파일을 확인해주세요.")
        st.stop()
    
    # 채팅 메시지 표시
    display_chat_messages()
    
    # 사용자 입력
    if prompt := st.chat_input("질문을 입력하세요..."):
        # 사용자 메시지 추가
        st.session_state.messages.append({"role": "user", "content": prompt})
        
        # 사용자 메시지 표시
        with st.chat_message("user"):
            st.markdown(prompt)
        
        # AI 응답 생성
        with st.chat_message("assistant"):
            with st.spinner("답변을 생성하고 있습니다..."):
                result = retrieve_and_generate(prompt, config)
                
                if result['success']:
                    st.markdown(result['answer'])
                    
                    # 소스 문서 표시
                    citations = result['citations']
                    if citations:
                        with st.expander(f"📚 참고 문서 ({len(citations)}개)"):
                            for i, citation in enumerate(citations, 1):
                                if 'retrievedReferences' in citation:
                                    for ref in citation['retrievedReferences']:
                                        location = ref.get('location', {})
                                        s3_location = location.get('s3Location', {})
                                        uri = s3_location.get('uri', 'Unknown')
                                        
                                        st.markdown(f"**{i}. {uri.split('/')[-1]}**")
                                        
                                        content = ref.get('content', {}).get('text', '')
                                        if content:
                                            st.markdown(f"```\n{content[:300]}...\n```")
                                        
                                        st.markdown("---")
                    
                    # 어시스턴트 메시지 저장
                    st.session_state.messages.append({
                        "role": "assistant", 
                        "content": result['answer'],
                        "citations": citations
                    })
                else:
                    st.error(result['answer'])
                    st.session_state.messages.append({
                        "role": "assistant", 
                        "content": result['answer']
                    })
'''

print("💬 메인 채팅 인터페이스 코드 완료")
print("포함된 기능:")
print("- RAG 검색 및 답변 생성")
print("- 실시간 채팅 인터페이스")
print("- 소스 문서 표시")
print("- 세션 기반 대화 관리")

💬 메인 채팅 인터페이스 코드 완료
포함된 기능:
- RAG 검색 및 답변 생성
- 실시간 채팅 인터페이스
- 소스 문서 표시
- 세션 기반 대화 관리


## 8. 파일 업로드 기능 

In [10]:
# 파일 업로드 기능 코드
file_upload_code = '''
def setup_file_upload():
    """파일 업로드 섹션 설정"""
    with st.sidebar:
        st.divider()
        st.subheader("📁 문서 업로드")
        
        uploaded_files = st.file_uploader(
            "문서를 업로드하세요",
            type=['pdf', 'txt', 'md', 'jpg', 'jpeg', 'png'],
            accept_multiple_files=True,
            help="PDF, 텍스트, 이미지 파일을 지원합니다"
        )
        
        if uploaded_files:
            st.success(f"{len(uploaded_files)}개 파일이 업로드되었습니다!")
            
            for file in uploaded_files:
                st.write(f"📄 {file.name} ({file.size:,} bytes)")
            
            if st.button("🚀 Knowledge Base에 추가", use_container_width=True):
                with st.spinner("파일을 처리하고 있습니다..."):
                    # TODO: 실제 파일 처리 및 S3 업로드 구현
                    st.success("파일이 성공적으로 추가되었습니다!")
                    st.info("Knowledge Base 동기화는 몇 분 정도 소요될 수 있습니다.")
        
        return uploaded_files
'''

print("📁 파일 업로드 기능 코드 완료")
print("지원 파일 형식: PDF, TXT, MD, JPG, JPEG, PNG")
print("다중 파일 업로드 지원")


📁 파일 업로드 기능 코드 완료
지원 파일 형식: PDF, TXT, MD, JPG, JPEG, PNG
다중 파일 업로드 지원


## 9. 완성된 app.py 생성

In [11]:
# 완성된 Streamlit 앱 코드 생성
complete_app_code = f'''
{streamlit_app_code}

{sidebar_code}

{chat_interface_code}

{file_upload_code}

# 메인 실행 함수
def main():
    """메인 애플리케이션 실행"""
    # 사이드바 설정
    config = setup_sidebar()
    
    # 파일 업로드
    uploaded_files = setup_file_upload()
    
    # 메인 채팅 인터페이스
    main_chat_interface(config)

if __name__ == "__main__":
    main()
'''

# app.py 파일로 저장
with open('app.py', 'w', encoding='utf-8') as f:
    f.write(complete_app_code)

print("🎉 app.py 파일이 생성되었습니다!")
print("\n실행 방법:")
print("streamlit run app.py")

# 파일 크기 확인
import os
file_size = os.path.getsize('app.py')
print(f"\n📊 생성된 파일 크기: {file_size:,} bytes ({file_size/1024:.1f} KB)")

🎉 app.py 파일이 생성되었습니다!

실행 방법:
streamlit run app.py

📊 생성된 파일 크기: 10,811 bytes (10.6 KB)


## 10. 테스트 및 실행

In [ ]:
# Streamlit 설정 파일 생성 (Jupyter 환경용)
import os
import subprocess
import time

# Streamlit 설정 디렉토리 생성
os.makedirs(os.path.expanduser('~/.streamlit'), exist_ok=True)

# 설정 파일 생성
config_content = """[server]
enableXsrfProtection = false
enableCORS = false
port = 8501
headless = true

[browser]
gatherUsageStats = false
"""

with open(os.path.expanduser('~/.streamlit/config.toml'), 'w') as f:
    f.write(config_content)

print("✅ Streamlit 설정 파일이 생성되었습니다.")

# Streamlit 앱 실행
def run_streamlit_app():
    """Streamlit 앱 실행"""
    if not os.path.exists('app.py'):
        print("❌ app.py 파일이 없습니다. 먼저 9번 섹션을 실행하세요.")
        return None
    
    print("🚀 Streamlit 앱을 시작합니다...")
    print("📱 브라우저에서 http://localhost:8501 로 접속하세요")
    
    try:
        # Streamlit 실행
        process = subprocess.Popen([
            'streamlit', 'run', 'app.py'
        ])
        
        print("✅ Streamlit 서버가 시작되었습니다!")
        print("🌐 URL: http://localhost:8501")
        print("⏹️ 중지하려면 Kernel을 재시작하세요")
        
        return process
        
    except Exception as e:
        print(f"❌ Streamlit 실행 실패: {e}")
        return None

# 실행
streamlit_process = run_streamlit_app()

print("\n📋 주요 기능:")
print("✅ 멀티모달 RAG 검색")
print("✅ Rerank 기반 고품질 답변")
print("✅ 실시간 채팅 인터페이스")
print("✅ 파일 업로드 (PDF, 텍스트, 이미지)")
print("✅ 소스 문서 추적 및 표시")
print("✅ 세션 기반 대화 관리")

✅ Streamlit 설정 파일이 생성되었습니다.
🚀 Streamlit 앱을 시작합니다...
📱 브라우저에서 http://localhost:8501 로 접속하세요
✅ Streamlit 서버가 시작되었습니다!
🌐 URL: http://localhost:8501
⏹️ 중지하려면 Kernel을 재시작하세요

📋 주요 기능:
✅ 멀티모달 RAG 검색
✅ Rerank 기반 고품질 답변
✅ 실시간 채팅 인터페이스
✅ 파일 업로드 (PDF, 텍스트, 이미지)
✅ 소스 문서 추적 및 표시
✅ 세션 기반 대화 관리


2025-08-25 04:19:07.617 Port 8501 is already in use
